In [1]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nn_ex"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

## Data: MNIST

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

## Simple Neural Network

In [4]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [10]:
reset_graph()


# Parameters/Variables/Placeholders

n_features = X_train.shape[1]
n_hidden1 = 100
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

# Dropout parameters
training = tf.placeholder_with_default(False, shape=(), name='training')
dropout_rate = 0  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)


# NN Model

from functools import partial

he_init = tf.variance_scaling_initializer()
#scale = 0.01
#kernel_regularizer = tf.contrib.layers.l1_regularizer(scale)
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=he_init) 
                            # option: kernel_regularizer= kernel_regularizer

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X_drop, n_hidden1, name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)   
    hidden2 = my_dense_layer(hidden1_drop, n_hidden2, name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)   
    logits = my_dense_layer(hidden1_drop, n_outputs, activation=None, name="outputs")
    

# Loss/Training/Evaluation

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy") 
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

with tf.name_scope("train"):
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [15]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 1000
batch_size = 20

# Early Stopping Parameters
max_checks_without_progress = 10
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./mnist_nn_model_01.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess,"./mnist_nn_model_01.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test*100))

0	Validation loss: 0.218353	Best loss: 0.218353	Accuracy: 94.72%
1	Validation loss: 0.279882	Best loss: 0.218353	Accuracy: 94.78%
2	Validation loss: 0.245630	Best loss: 0.218353	Accuracy: 95.72%
3	Validation loss: 0.262549	Best loss: 0.218353	Accuracy: 95.64%
4	Validation loss: 0.272777	Best loss: 0.218353	Accuracy: 96.16%
5	Validation loss: 0.316229	Best loss: 0.218353	Accuracy: 95.66%
6	Validation loss: 0.353235	Best loss: 0.218353	Accuracy: 96.38%
7	Validation loss: 0.328619	Best loss: 0.218353	Accuracy: 96.18%
8	Validation loss: 0.491521	Best loss: 0.218353	Accuracy: 95.38%
9	Validation loss: 0.421706	Best loss: 0.218353	Accuracy: 96.44%
10	Validation loss: 0.387184	Best loss: 0.218353	Accuracy: 96.64%
Early stopping!
INFO:tensorflow:Restoring parameters from ./mnist_nn_model_01.ckpt
Final test accuracy: 94.55%


### 0-4 MNIST data

In [24]:
id_train04 = (y_train < 5)
id_val04 = (y_valid < 5)
id_test04 = (y_test < 5)

X_train04 = X_train[id_train04]
y_train04 = y_train[id_train04]
X_val04 = X_valid[id_val04]
y_val04 = y_valid[id_val04]
X_test04 = X_test[id_test04]
y_test04 = y_test[id_test04]

In [25]:
print(X_train04.shape, y_train04.shape)
print(X_val04.shape, y_val04.shape)
print(X_test04.shape, y_test04.shape)

(28038, 784) (28038,)
(2558, 784) (2558,)
(5139, 784) (5139,)


## Deeper model
(MNIST 0-4 digits)

In [16]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers, n_neurons, name=None, 
        activation = tf.nn.elu, initializer= he_init):
    with tf.variable_scope(name,'dnn'):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs,n_neurons, activation=activation, 
                                     kernel_initializer = initializer,
                                    name = "hidden%d" % (layer+1))
        return inputs

In [26]:
reset_graph()


# Parameters/Variables/Placeholders

n_features = X_train04.shape[1]
n_hidden = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")


# NN Model

with tf.name_scope("dnn"):
    dnn_outputs = dnn(X, n_hidden_layers=5, n_neurons =n_hidden) 
    logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="outputs")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    

# Loss/Training/Evaluation

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy") 
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

with tf.name_scope("train"):
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [27]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 1000
batch_size = 20

# Early Stopping Parameters
max_checks_without_progress = 10
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train04, y_train04, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val04, y: y_val04})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./mnist_nn_model_01.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess,"./mnist_nn_model_01.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test04, y: y_test04})
    print("Final test accuracy: {:.2f}%".format(acc_test*100))

0	Validation loss: 0.116407	Best loss: 0.116407	Accuracy: 97.58%
1	Validation loss: 0.180534	Best loss: 0.116407	Accuracy: 97.11%
2	Validation loss: 0.227535	Best loss: 0.116407	Accuracy: 93.86%
3	Validation loss: 0.107346	Best loss: 0.107346	Accuracy: 97.54%
4	Validation loss: 0.302668	Best loss: 0.107346	Accuracy: 95.35%
5	Validation loss: 1.631054	Best loss: 0.107346	Accuracy: 22.01%
6	Validation loss: 1.635262	Best loss: 0.107346	Accuracy: 18.73%
7	Validation loss: 1.671200	Best loss: 0.107346	Accuracy: 22.01%
8	Validation loss: 1.695277	Best loss: 0.107346	Accuracy: 19.27%
9	Validation loss: 1.744607	Best loss: 0.107346	Accuracy: 20.91%
10	Validation loss: 1.629857	Best loss: 0.107346	Accuracy: 22.01%
11	Validation loss: 1.810803	Best loss: 0.107346	Accuracy: 22.01%
12	Validation loss: 1.675703	Best loss: 0.107346	Accuracy: 18.73%
13	Validation loss: 1.633233	Best loss: 0.107346	Accuracy: 20.91%
Early stopping!
INFO:tensorflow:Restoring parameters from ./mnist_nn_model_01.ckpt
Fin

## DNN with hyperparameter tuning

We create a `DNNClassifier` class compatible with Scikit-Learn's `RandomizedSearchCV` class, to perform hyperparameter tuning.

In [33]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 10
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

Testing the DNNClassifier

In [34]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train04, y_train04, n_epochs=1000, X_valid=X_val04, y_valid=y_val04)

0	Validation loss: 0.116407	Best loss: 0.116407	Accuracy: 97.58%
1	Validation loss: 0.180534	Best loss: 0.116407	Accuracy: 97.11%
2	Validation loss: 0.227535	Best loss: 0.116407	Accuracy: 93.86%
3	Validation loss: 0.107346	Best loss: 0.107346	Accuracy: 97.54%
4	Validation loss: 0.302668	Best loss: 0.107346	Accuracy: 95.35%
5	Validation loss: 1.631054	Best loss: 0.107346	Accuracy: 22.01%
6	Validation loss: 1.635262	Best loss: 0.107346	Accuracy: 18.73%
7	Validation loss: 1.671200	Best loss: 0.107346	Accuracy: 22.01%
8	Validation loss: 1.695277	Best loss: 0.107346	Accuracy: 19.27%
9	Validation loss: 1.744607	Best loss: 0.107346	Accuracy: 20.91%
10	Validation loss: 1.629857	Best loss: 0.107346	Accuracy: 22.01%
11	Validation loss: 1.810803	Best loss: 0.107346	Accuracy: 22.01%
12	Validation loss: 1.675703	Best loss: 0.107346	Accuracy: 18.73%
13	Validation loss: 1.633233	Best loss: 0.107346	Accuracy: 20.91%
14	Validation loss: 1.652905	Best loss: 0.107346	Accuracy: 20.91%
Early stopping!


DNNClassifier(activation=<function elu at 0x125c65268>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0xb305edf28>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [35]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test04)
accuracy_score(y_test04, y_pred)

0.9725627553998832

Now tuning hyperparameters

In [37]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [50, 100, 150],
    #"batch_size": [10, 50, 100],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    "n_hidden_layers": [2, 4, 6],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train04, y_train04, X_valid=X_val04, y_valid=y_val04, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=50, n_hidden_layers=2, learning_rate=0.02, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0xb2d8c7620> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.249434	Best loss: 0.249434	Accuracy: 95.04%
1	Validation loss: 0.123172	Best loss: 0.123172	Accuracy: 96.83%
2	Validation loss: 0.243961	Best loss: 0.123172	Accuracy: 96.87%
3	Validation loss: 0.302742	Best loss: 0.123172	Accuracy: 95.54%
4	Validation loss: 0.202096	Best loss: 0.123172	Accuracy: 97.26%
5	Validation loss: 0.289979	Best loss: 0.123172	Accuracy: 97.34%
6	Validation loss: 0.492861	Best loss: 0.123172	Accuracy: 96.95%
7	Validation loss: 0.518559	Best loss: 0.123172	Accuracy: 96.91%
8	Validation loss: 0.331733	Best loss: 0.123172	Accuracy: 97.69%
9	Validation loss: 0.553148	Best loss: 0.123172	Accuracy: 97.54%
10	Validation loss: 0.611440	Best loss: 0.123172	Accuracy: 96.99%
11	Validation loss: 0.572985	Best loss: 0.123172	Accuracy: 97.30%
12	Validation loss: 0.868629	Best loss: 0.123172	Accuracy: 97.26%
Early stopping!
[CV]  n_neurons=50, n_hidden_layers=2, learning_rate=0.02, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0xb2d8c76

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.7s remaining:    0.0s


0	Validation loss: 0.228452	Best loss: 0.228452	Accuracy: 95.93%
1	Validation loss: 0.136174	Best loss: 0.136174	Accuracy: 97.30%
2	Validation loss: 0.185548	Best loss: 0.136174	Accuracy: 96.83%
3	Validation loss: 0.320776	Best loss: 0.136174	Accuracy: 96.36%
4	Validation loss: 0.230437	Best loss: 0.136174	Accuracy: 96.95%
5	Validation loss: 0.360208	Best loss: 0.136174	Accuracy: 97.26%
6	Validation loss: 0.397652	Best loss: 0.136174	Accuracy: 96.21%
7	Validation loss: 0.370255	Best loss: 0.136174	Accuracy: 97.42%
8	Validation loss: 0.457577	Best loss: 0.136174	Accuracy: 98.12%
9	Validation loss: 0.414796	Best loss: 0.136174	Accuracy: 97.77%
10	Validation loss: 0.729730	Best loss: 0.136174	Accuracy: 96.64%
11	Validation loss: 0.519125	Best loss: 0.136174	Accuracy: 97.58%
12	Validation loss: 0.834429	Best loss: 0.136174	Accuracy: 97.73%
Early stopping!
[CV]  n_neurons=50, n_hidden_layers=2, learning_rate=0.02, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0xb2d8c76

...

7	Validation loss: 1.612707	Best loss: 1.609777	Accuracy: 18.73%
8	Validation loss: 1.612845	Best loss: 1.609777	Accuracy: 19.27%
9	Validation loss: 1.611299	Best loss: 1.609777	Accuracy: 22.01%
10	Validation loss: 1.610328	Best loss: 1.609777	Accuracy: 19.27%
11	Validation loss: 1.611546	Best loss: 1.609777	Accuracy: 22.01%
12	Validation loss: 1.612742	Best loss: 1.609777	Accuracy: 19.27%
13	Validation loss: 1.613987	Best loss: 1.609777	Accuracy: 19.08%
Early stopping!
[CV]  n_neurons=50, n_hidden_layers=6, learning_rate=0.05, activation=<function relu at 0x125c7e598>, total=  15.9s
[CV] n_neurons=50, n_hidden_layers=6, learning_rate=0.05, activation=<function relu at 0x125c7e598> 
0	Validation loss: 1.208835	Best loss: 1.208835	Accuracy: 39.29%
1	Validation loss: 1.180306	Best loss: 1.180306	Accuracy: 35.81%
2	Validation loss: 1.616475	Best loss: 1.180306	Accuracy: 19.27%
3	Validation loss: 1.622266	Best loss: 1.180306	Accuracy: 22.01%
4	Validation loss: 1.608986	Best loss: 1.180306	

33	Validation loss: 0.779403	Best loss: 0.767368	Accuracy: 62.04%
34	Validation loss: 0.773933	Best loss: 0.767368	Accuracy: 59.30%
35	Validation loss: 0.800630	Best loss: 0.767368	Accuracy: 60.79%
Early stopping!
[CV]  n_neurons=150, n_hidden_layers=4, learning_rate=0.05, activation=<function relu at 0x125c7e598>, total= 1.2min
[CV] n_neurons=150, n_hidden_layers=4, learning_rate=0.05, activation=<function relu at 0x125c7e598> 
0	Validation loss: 1.668997	Best loss: 1.668997	Accuracy: 21.97%
1	Validation loss: 1.618384	Best loss: 1.618384	Accuracy: 19.27%
2	Validation loss: 1.613710	Best loss: 1.613710	Accuracy: 19.27%
3	Validation loss: 1.622370	Best loss: 1.613710	Accuracy: 22.01%
4	Validation loss: 1.608999	Best loss: 1.608999	Accuracy: 22.01%
5	Validation loss: 1.616001	Best loss: 1.608999	Accuracy: 22.01%
6	Validation loss: 1.615687	Best loss: 1.608999	Accuracy: 19.27%
7	Validation loss: 1.612685	Best loss: 1.608999	Accuracy: 22.01%
8	Validation loss: 1.613897	Best loss: 1.608999

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 55.1min finished


0	Validation loss: 0.112554	Best loss: 0.112554	Accuracy: 97.34%
1	Validation loss: 0.086099	Best loss: 0.086099	Accuracy: 97.69%
2	Validation loss: 0.130937	Best loss: 0.086099	Accuracy: 98.05%
3	Validation loss: 0.069808	Best loss: 0.069808	Accuracy: 98.20%
4	Validation loss: 0.060656	Best loss: 0.060656	Accuracy: 98.83%
5	Validation loss: 0.098828	Best loss: 0.060656	Accuracy: 97.89%
6	Validation loss: 0.099327	Best loss: 0.060656	Accuracy: 98.05%
7	Validation loss: 0.092333	Best loss: 0.060656	Accuracy: 98.44%
8	Validation loss: 0.092289	Best loss: 0.060656	Accuracy: 98.08%
9	Validation loss: 0.057853	Best loss: 0.057853	Accuracy: 98.51%
10	Validation loss: 0.386360	Best loss: 0.057853	Accuracy: 97.93%
11	Validation loss: 0.143013	Best loss: 0.057853	Accuracy: 98.48%
12	Validation loss: 0.117396	Best loss: 0.057853	Accuracy: 98.44%
13	Validation loss: 0.175498	Best loss: 0.057853	Accuracy: 98.05%
14	Validation loss: 0.089757	Best loss: 0.057853	Accuracy: 98.59%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DNNClassifier(activation=<function elu at 0x125c65268>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0xb305edf28>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid='warn', n_iter=50, n_jobs=None,
          param_distributions={'n_neurons': [50, 100, 150], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x125c7e598>, <function elu at 0x125c65268>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0xb2d8c79d8>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0xb2d8c7620>], 'n_hidden_layers': [2, 4, 6]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [38]:
rnd_search.best_params_

{'activation': <function __main__.leaky_relu.<locals>.parametrized_leaky_relu>,
 'learning_rate': 0.01,
 'n_hidden_layers': 2,
 'n_neurons': 100}

In [40]:
y_pred = rnd_search.predict(X_test04)
accuracy_score(y_test04, y_pred)

0.9859894921190894

In [41]:
rnd_search.best_estimator_.save("./mnist04_best_dnn_model")